In [1]:
###script to preprocess the pole data based on the R script that I created


In [15]:
import numpy as np
import pandas as pd
import re

dems_2016 = pd.read_csv("../data/democrats2016.csv")
reps_2016 = pd.read_csv("../data/republicans2016.csv")
dems = dems_2016.copy()

# filter down to second date
dems["Date"].str.replace(".{3,} - ", "")

def something(date_string):
    second_char = date_string[1]
    if second_char == "/":
        month = date_string[0:1]
    else:
        month = date_string[0:2]
    

0        6/5
1        6/5
2       5/30
3       5/19
4       5/19
5       5/17
6        5/1
7       4/28
8       4/24
9       4/19
10      4/13
11      4/14
12      4/12
13       4/3
14       4/2
15      3/31
16      3/27
17      3/26
18      3/22
19      3/22
20      3/20
21      3/20
22      3/20
23      3/21
24       3/6
25       3/6
26      2/27
27      2/28
28      2/24
29      2/17
       ...  
138     9/29
139     7/22
140     7/20
141     6/30
142      6/1
143     4/15
144     3/30
145      3/9
146      3/9
147     1/26
148     1/23
149     1/19
150    12/16
151    12/15
152     12/9
153     12/5
154    11/20
155     11/8
156    10/31
157     9/29
158      9/8
159      8/2
160     7/21
161     7/18
162      5/9
163     4/29
164     3/30
165      2/3
166      1/6
167     12/2
Name: Date, Length: 168, dtype: object

In [ ]:
def preprocess_dates(df):
    '''takes a data frame with a "Poll" column and processes it to include the final date. 
    useful for analyzing poll data in the context of this project
    '''
    # first part: we only have month - date, with no year
    # months and days aren't in consistent format so we need to hack that together too
    df.Poll.str.